# 0. Install and Import Dependencies

In [1]:
!pip3 install mediapipe opencv-python
!pip3 install numpy
!pip3 install tensorflow


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
def calc_angle(p1, p2, p3):
    a = np.array([p1.x, p1.y,p1.z])
    b = np.array([p2.x, p2.y, p2.z])
    c = np.array([p3.x, p3.y,p3.z])

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.rad2deg(np.arccos(cosine_angle))

    return angle

In [4]:
def calc_dist(p1, p2):
    a = np.array([p1.x, p1.y,p1.z])
    b = np.array([p2.x, p2.y, p2.z])

    squared_dist = np.sum((a-b)**2, axis=0)
    return np.sqrt(squared_dist)

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

# 4. Curl Counter

In [5]:
curr_pos = 'down'
reps = 0

def knee_extension(landmarks):
    global curr_pos, reps
    next_pos = ""
    standing = False
    alreadyRight = False

    left_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
    right_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

    left_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
    right_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]

    left_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

    left_knee_angle = calc_angle(left_ankle, left_knee, left_hip)
    right_knee_angle = calc_angle(right_ankle, right_knee, right_hip)
    
    errors = []
    leg_down = "left"

    if right_knee_angle >= 135 and left_knee_angle >= 135:
        return None

    if left_ankle.y < right_ankle.y:
        leg_down = "right"
    
    if leg_down == "right":
        reps += 1
        alreadyRight = True
        if right_knee_angle < 140:
            errors.append("Don't bend right leg")
        if abs(left_hip.y - left_knee.y) > 0.05:
            errors.append("Keep left knee parallel to hip")
        if left_knee_angle >= 160:
            standing = True
        if left_knee_angle >= 130 and not standing:
            next_pos = "up"
        if left_knee_angle < 70 and not standing:
            curr_pos = "down"


    if leg_down == "left":
        reps+=1
        if left_knee_angle < 140:
            errors.append("Don't bend left leg")
        if abs (right_hip.y - right_knee.y) > 0.05:
            errors.append("Keep right knee parallel to hip")
        if right_knee_angle >= 160:
            standing = True
        if right_knee_angle >= 130 and not standing and not alreadyRight:
            next_pos = "up"
        if right_knee_angle < 70 and not standing and not alreadyRight:
            curr_pos = "down"
    
    
    if next_pos == "up" and curr_pos == "down":
         reps += 1
         curr_pos = "up"
    if curr_pos == "up" and next_pos == "down":
         curr_pos = "down"
    
    return errors

In [23]:
curr_pos = 'down'
reps = 0
def dumbell_thrust(landmarks):
    global curr_pos, reps
    next_pos = ""
    errors = []

    left_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
    right_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

    left_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
    right_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]

    left_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

    left_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

    left_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

    left_knee_angle = calc_angle(left_ankle, left_knee, left_hip)
    right_knee_angle = calc_angle(right_ankle, right_knee, right_hip)

    if (left_knee_angle < 80) and (right_knee_angle < 80) :
         next_pos = "down"
    if (left_knee_angle > 130) and (right_knee_angle > 130) :
         next_pos = "up"

    if left_knee_angle < 90 or right_knee_angle < 90:
        errors.append("Don't bend down too much")
    
    if abs(calc_dist(left_ankle, right_ankle) - calc_dist(left_hip, right_hip)) > 0.1:
        errors.append("Keep legs hip width apart")
    
    if abs(calc_dist(left_elbow, right_elbow) - calc_dist(left_shoulder, right_shoulder)) > 0.1:
        errors.append("Keep elbows parallel to shoulders")

    if left_elbow.z < left_shoulder.z + .1 or right_elbow.z < right_shoulder.z + .05:
        errors.append("Don't bend elbows down too much")
    if next_pos == "up" and curr_pos == "down":
         reps += 1
         curr_pos = "up"
    if curr_pos == "up" and next_pos == "down":
         curr_pos = "down"

    return errors

In [7]:
def external_rotation(landmarks):

    errors = []
    active_arm = "right"

    left_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

    left_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

    left_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

    left_arm_angle = calc_angle(left_elbow, left_wrist, left_shoulder)
    right_arm_angle = calc_angle(right_elbow, right_wrist, right_shoulder)

    
    if left_elbow.z > right_elbow.z:
        active_arm = "left"
    
    if active_arm == "right":
        if right_arm_angle < 90:
            errors.append("Angle in right arm is less than 90 degrees")
        if right_arm_angle > 120:
            errors.append("Angle in right arm is more than 90 degrees")
        if right_wrist.z > right_elbow.z:
            errors.append("Keep right wrist perpendicular to elbow")
        

    if active_arm == "left":
        if left_arm_angle < 90:
            errors.append("Angle in left arm is less than 90 degrees")
        if calc_angle(left_elbow, left_wrist, left_shoulder) > 120:
            errors.append("Angle in left arm is more than 90 degrees")
        if left_wrist.z > left_elbow.z:
            errors.append("Keep left wrist perpendicular to elbow")
    
    
    return errors

In [8]:
def rotator_cuff(landmarks):
    errors = []
    active_arm = "right"

    left_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

    left_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

    left_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

    left_arm_angle = calc_angle(left_elbow, left_wrist, left_shoulder)
    right_arm_angle = calc_angle(right_elbow, right_wrist, right_shoulder)

    if left_elbow.z > right_elbow.z:
        active_arm = "left"
    
    if active_arm == "right":
        if right_arm_angle < 70:
            errors.append("Angle in right arm is less than 90 degrees")
        if right_arm_angle > 110:
            errors.append("Angle in right arm is greater than 90 degrees")
        if abs(right_elbow.z - right_shoulder.z) > .1:
            errors.append("Keep right elbow perpendicular to shoulder")
        

    if active_arm == "left":
        if left_arm_angle < 70:
            errors.append("Angle in left arm is less than 90 degrees")
        if left_arm_angle > 110:
            errors.append("Angle in left arm is greater than 90 degrees")
        if abs(left_elbow.z - left_shoulder.z) > .1:
            errors.append("Keep left elbow perpendicular to shoulder")
            
    return errors

In [ ]:
import traceback
reps = 0

# Initialize VideoCapture
cap = cv2.VideoCapture(0)

# Setup mediapipe instance
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
# Main loop
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic, mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Recolor image to RGB
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            errors = dumbell_thrust(landmarks)
            for i in range(len(errors)):
                cv2.putText(image, f'Error: {errors[i]}', (50,50 + 80*i), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)

            # cv2.putText(image, f'Info: {", ".join(errors)}', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 2)
            cv2.putText(image, f'Reps: {reps}', (1500,100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
        except:
            print(traceback.format_exc())

        # Make detection
        results = pose.process(image)
        results_face = holistic.process(image)

        #mp_drawing.draw_landmarks(image, results_face.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )               
        
        cv2.imshow('Dumbell Thrust Analysis', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release VideoCapture and close all windows
cap.release()
cv2.destroyAllWindows()
            